In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
# DB libraries
import psycopg2
from postgres_data import host
from postgres_data import password
from postgres_data import user
from sqlalchemy import create_engine

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'actor_name',
    'actor_id', 'character', 'actor_gender', 'actor_popularity', 'director_name', 'director_id', 
    'director_gender', 'director_popularity', 'studio_id', 'studio_name', 'origin_country'
]

target = ['revenue']

In [6]:
# Create an engine instance
alchemyEngine = create_engine(f'postgresql://{user}:{password}@{host}:5432/Movie_Data')

In [7]:
# Connect to PostgreSQL server
dbConnection = alchemyEngine.connect()

# Read data from PostgreSQL database table and load into a DataFrame instance
df = pd.read_sql("select * from meta", dbConnection)
dbConnection.close()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.actor_gender[df.actor_gender == 2] = 'male'
df.actor_gender[df.actor_gender == 1] = 'female'

df.director_gender[df.director_gender == 2] = 'male'
df.director_gender[df.director_gender == 1] = 'female'

df

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_name,...,actor_gender,character,actor_popularity,director_name,director_id,director_gender,director_popularity,studio_id,studio_name,origin_country
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,...,female,Hermione Granger,16.930,Chris Columbus,10965.0,male,2.589,436.0,1492 Pictures,US
1,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,...,female,Hermione Granger,16.930,Chris Columbus,10965.0,male,2.589,174.0,Warner Bros. Pictures,US
2,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965.0,male,2.589,436.0,1492 Pictures,US
3,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965.0,male,2.589,437.0,Heyday Films,GB
4,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965.0,male,2.589,174.0,Warner Bros. Pictures,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,...,male,Dr. Peter Venkman,14.048,Jason Reitman,52443.0,male,4.105,13240.0,Bron Studios,CA
1265,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,...,female,Callie Spengler,11.375,Jason Reitman,52443.0,male,4.105,34.0,Sony Pictures,US
1266,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,...,female,Callie Spengler,11.375,Jason Reitman,52443.0,male,4.105,5.0,Columbia Pictures,US
1267,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Finn Wolfhard,...,male,Trevor,17.350,Jason Reitman,52443.0,male,4.105,84042.0,Ghost Corps,US


In [8]:
# More data cleaning
noID_df = df.drop(columns=['movie_id', 'actor_id', 'director_id', 'studio_id'])

noID_df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_name,actor_gender,character,actor_popularity,director_name,director_gender,director_popularity,studio_name,origin_country
0,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,1492 Pictures,US
1,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,Warner Bros. Pictures,US
2,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,1492 Pictures,US
3,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Heyday Films,GB
4,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Warner Bros. Pictures,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,male,Dr. Peter Venkman,14.048,Jason Reitman,male,4.105,Bron Studios,CA
1265,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,female,Callie Spengler,11.375,Jason Reitman,male,4.105,Sony Pictures,US
1266,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,female,Callie Spengler,11.375,Jason Reitman,male,4.105,Columbia Pictures,US
1267,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Finn Wolfhard,male,Trevor,17.350,Jason Reitman,male,4.105,Ghost Corps,US


In [9]:
# More data cleaning
noID_df = df.drop(columns=['movie_id', 'actor_id', 'director_id', 'studio_id'])

noID_df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_name,actor_gender,character,actor_popularity,director_name,director_gender,director_popularity,studio_name,origin_country
0,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,1492 Pictures,US
1,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,Warner Bros. Pictures,US
2,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,1492 Pictures,US
3,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Heyday Films,GB
4,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Warner Bros. Pictures,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,male,Dr. Peter Venkman,14.048,Jason Reitman,male,4.105,Bron Studios,CA
1265,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,female,Callie Spengler,11.375,Jason Reitman,male,4.105,Sony Pictures,US
1266,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,female,Callie Spengler,11.375,Jason Reitman,male,4.105,Columbia Pictures,US
1267,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Finn Wolfhard,male,Trevor,17.350,Jason Reitman,male,4.105,Ghost Corps,US


In [10]:
# # Changing to date and time
# from datetime import datetime

# noID_df['release_date'] = pd.to_datetime(noID_df['release_date'])

# noID_df

In [11]:
# trying out grouping
grouped_df = noID_df.groupby('title')

grouped_lists = grouped_df['studio_name'].agg(lambda column: ", ".join(column))

grouped_lists = grouped_lists.reset_index(name='studio_name')

grouped_lists

,title,studio_name
0,A Quiet Place Part II,"Paramount, Platinum Dunes, Platinum Dunes, Sun..."
1,After We Fell,"CalMaple Films, Vertical Entertainment, CalMap..."
2,Avatar,"20th Century Fox, 20th Century Fox, Ingenious ..."
3,Avengers: Endgame,"Marvel Studios, Marvel Studios, Marvel Studios..."
4,Avengers: Infinity War,"Marvel Studios, Marvel Studios, Marvel Studios..."
...,...,...
62,Venom,"Pascal Pictures, Matt Tolmach Productions, Avi..."
63,Venom: Let There Be Carnage,"Sony Pictures, Columbia Pictures, Pascal Pictu..."
64,Wonder Woman 1984,"DC Entertainment, Atlas Entertainment, DC Comi..."
65,Wrath of Man,"Metro-Goldwyn-Mayer, Lionsgate, CAA Media Fina..."


In [12]:
# Create our features
X = pd.get_dummies(noID_df.drop(columns='revenue'))


# Create our target
y = pd.get_dummies(noID_df['revenue'])

In [13]:
X.describe()

,movie_popularity,vote_average,vote_count,budget,actor_popularity,director_popularity,title_A Quiet Place Part II,title_After We Fell,title_Avatar,title_Avengers: Endgame,...,studio_name_Zero Gravity Management,studio_name_ufotable,origin_country_AU,origin_country_CA,origin_country_CN,origin_country_DE,origin_country_FR,origin_country_GB,origin_country_JP,origin_country_US
count,1167.000000,1167.000000,1167.000000,1.167000e+03,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,...,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000
mean,779.177652,7.608912,9034.667524,1.376702e+08,18.330002,4.504798,0.005141,0.013710,0.013710,0.047986,...,0.001714,0.001714,0.005141,0.008569,0.010283,0.005141,0.016281,0.059983,0.008569,0.886033
std,1208.215990,0.643057,7243.874912,9.012646e+07,9.925773,3.662854,0.071550,0.116336,0.116336,0.213829,...,0.041380,0.041380,0.071550,0.092211,0.100925,0.071550,0.126609,0.237557,0.092211,0.317908
min,173.026000,6.000000,40.000000,1.000000e+07,10.005000,0.728000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,256.019000,7.200000,2347.000000,7.000000e+07,11.672000,1.836000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,391.146000,7.700000,6850.000000,1.160000e+08,14.904000,2.915000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,672.641000,8.300000,15236.000000,2.000000e+08,21.049000,6.150000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,8304.971000,8.400000,25788.000000,3.560000e+08,60.132000,16.390000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
# Check the balance of our target values
# y['loan_status'].value_counts()
y = noID_df['revenue']
y.value_counts()

657000000     104
375540831      84
2797800564     56
2046239637     42
708000000      40
             ... 
61779301        4
148810604       3
112000000       2
467863133       2
136384442       2
Name: revenue, Length: 67, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
Counter(y_train)

Counter({375540831: 63,
         757930663: 7,
         233274812: 3,
         954305868: 9,
         657000000: 78,
         331096766: 21,
         789804554: 13,
         16000000: 26,
         721077945: 16,
         467863133: 2,
         821708551: 15,
         2046239637: 31,
         708000000: 30,
         167381210: 30,
         880166924: 26,
         2797800564: 42,
         165160005: 9,
         933959197: 8,
         213644366: 18,
         708962323: 11,
         503063688: 6,
         34560577: 13,
         876688482: 9,
         132000000: 3,
         168285000: 6,
         938212738: 10,
         976475550: 7,
         153000000: 11,
         300000000: 5,
         442000000: 22,
         890871626: 8,
         378328978: 4,
         297372261: 5,
         629443428: 18,
         36964325: 8,
         25814306: 5,
         2847246203: 12,
         430238384: 8,
         623933331: 3,
         175302354: 12,
         348319861: 13,
         4700000: 18,
         15544

# Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({375540831: 78,
         757930663: 78,
         233274812: 78,
         954305868: 78,
         657000000: 78,
         331096766: 78,
         789804554: 78,
         16000000: 78,
         721077945: 78,
         467863133: 78,
         821708551: 78,
         2046239637: 78,
         708000000: 78,
         167381210: 78,
         880166924: 78,
         2797800564: 78,
         165160005: 78,
         933959197: 78,
         213644366: 78,
         708962323: 78,
         503063688: 78,
         34560577: 78,
         876688482: 78,
         132000000: 78,
         168285000: 78,
         938212738: 78,
         976475550: 78,
         153000000: 78,
         300000000: 78,
         442000000: 78,
         890871626: 78,
         378328978: 78,
         297372261: 78,
         629443428: 78,
         36964325: 78,
         25814306: 78,
         2847246203: 78,
         430238384: 78,
         623933331: 78,
         175302354: 78,
         348319861: 78,
         4700000:

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 0,  0,  0, ...,  0,  8,  0],
       [ 0,  0,  0, ...,  0,  6,  0],
       [ 0,  0,  0, ...,  0,  9,  0],
       ...,
       [ 0,  0,  0, ...,  0, 11,  0],
       [ 0,  0,  0, ...,  0, 14,  0],
       [ 0,  0,  0, ...,  0,  4,  0]], dtype=int64)

In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.015625

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     344931       0.00      0.00      1.00      0.00      0.00      0.00         8
    4700000       0.00      0.00      1.00      0.00      0.00      0.00         6
   16000000       0.00      0.00      1.00      0.00      0.00      0.00         9
   17635215       0.00      0.00      1.00      0.00      0.00      0.00         2
   19000000       0.00      0.00      1.00      0.00      0.00      0.00         4
   22039969       0.00      0.00      1.00      0.00      0.00      0.00         4
   24640084       0.00      0.00      1.00      0.00      0.00      0.00         2
   25814306       0.00      0.00      1.00      0.00      0.00      0.00         2
   34560577       0.00      0.00      1.00      0.00      0.00      0.00         5
   36964325       0.00      0.00      1.00      0.00      0.00      0.00         2
   45000000       0.00      0.00      1.00      0.00      0.00      0.00         1
   

# Undersampling

In [21]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({344931: 2,
         4700000: 2,
         16000000: 2,
         17635215: 2,
         19000000: 2,
         22039969: 2,
         24640084: 2,
         25814306: 2,
         34560577: 2,
         36964325: 2,
         45000000: 2,
         61779301: 2,
         89500000: 2,
         103966489: 2,
         112000000: 2,
         127000000: 2,
         130000000: 2,
         132000000: 2,
         136384442: 2,
         148810604: 2,
         153000000: 2,
         155446362: 2,
         165160005: 2,
         167381210: 2,
         168285000: 2,
         175302354: 2,
         199166992: 2,
         213644366: 2,
         233274812: 2,
         297372261: 2,
         300000000: 2,
         331096766: 2,
         348319861: 2,
         354264482: 2,
         375540831: 2,
         378328978: 2,
         430238384: 2,
         442000000: 2,
         467863133: 2,
         503063688: 2,
         623933331: 2,
         629443428: 2,
         657000000: 2,
         708000000: 2,
    

In [22]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.015625

In [24]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 0,  0,  0, ...,  0,  8,  0],
       [ 0,  0,  0, ...,  0,  6,  0],
       [ 0,  0,  0, ...,  0,  9,  0],
       ...,
       [ 0,  0,  0, ...,  0, 11,  0],
       [ 0,  0,  0, ...,  0, 14,  0],
       [ 0,  0,  0, ...,  0,  4,  0]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     344931       0.00      0.00      1.00      0.00      0.00      0.00         8
    4700000       0.00      0.00      1.00      0.00      0.00      0.00         6
   16000000       0.00      0.00      1.00      0.00      0.00      0.00         9
   17635215       0.00      0.00      1.00      0.00      0.00      0.00         2
   19000000       0.00      0.00      1.00      0.00      0.00      0.00         4
   22039969       0.00      0.00      1.00      0.00      0.00      0.00         4
   24640084       0.00      0.00      1.00      0.00      0.00      0.00         2
   25814306       0.00      0.00      1.00      0.00      0.00      0.00         2
   34560577       0.00      0.00      1.00      0.00      0.00      0.00         5
   36964325       0.00      0.00      1.00      0.00      0.00      0.00         2
   45000000       0.00      0.00      1.00      0.00      0.00      0.00         1
   